# Бустинг. Практика

 В предыдущем юните мы разобрали ансамблевый метод под названием бустинг, а также рассмотрели различные его вариации. В этом юните мы решим практический кейс с использованием данного алгоритма, а также сравним изученные модификации бустинга.

Мы будем работать c данными, которые содержат результаты опроса клиентов авиакомпании по поводу их удовлетворённости полётом.

Нашей задачей будет предсказать удовлетворённость пассажиров.

# Признаки в данных
- Gender — пол пассажира (женский, мужской)
- Customer Type — тип клиента (постоянный/непостоянный клиент)
- Age — возраст клиента
- Type of Travel — цель перелета (личная/деловая поездка)
- Class — туристический класс пассажира (Business, Eco, Eco Plus)
- Flight distance — расстояние полета
- Inflight wifi service — уровень удовлетворённости Wi-Fi (0 — не применимо, 1–5)
- Departure/Arrival time convenient — уровень удовлетворённости временем отправления и прибытия
- Ease of Online booking — уровень удовлетворённости онлайн-бронированием
- Gate location — уровень удовлетворённости расположением выхода на посадку
- Food and drink — уровень удовлетворённости едой и напитками
- Online boarding — уровень удовлетворённости онлайн-регистрацией
- Seat comfort — уровень удовлетворённости комфортом сидений
- Inflight entertainment — уровень удовлетворённости развлечениями на борту
- On-board service — уровень удовлетворённости сервисом на борту
- Leg room service — уровень удовлетворённости местом для ног
- Baggage handling — уровень удовлетворённости обработкой багажа
- Check-in service — уровень удовлетворённости услугами регистрации
- Inflight service — уровень удовлетворённости обслуживанием во время полёта
- Cleanliness — уровень удовлетворённости чистотой
- Departure Delay in Minutes — задержка при отправлении (в минутах)
- Arrival Delay in Minutes — задержка при прибытии (в минутах)
- Satisfaction — удовлетворённость авиакомпанией — целевая переменная (satisfaction/neutral/dissatisfaction).

In [57]:
import numpy as np
import pandas as pd
from sklearn import linear_model #линейные моделиё
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from catboost import Pool, CatBoostClassifier
from catboost.utils import get_confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [36]:
df=pd.read_csv('AirPass.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


ЗАДАНИЕ 6.1

In [37]:
df.isnull().sum().sum()

310

ЗАДАНИЕ 6.2

In [38]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
df['Arrival Delay in Minutes'].mean()

15.133392362180475

ЗАДАНИЕ 6.3

In [39]:
print(df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0])
print(df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0])
print(df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0])

Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64

In [40]:
df.satisfaction = df.satisfaction.map({'neutral or dissatisfied': 0, 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1 , 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0 , 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0 , 'Female': 1})

ЗАДАНИЕ 6.4

In [41]:
df=pd.get_dummies(df)
df.shape

(103904, 27)

ЗАДАНИЕ 6.5

In [42]:
X=df.drop('satisfaction', axis=1)
y =df['satisfaction']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

ЗАДАНИЕ 6.6

In [44]:
scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]


1.193295224074621

ЗАДАНИЕ 6.7

In [45]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
f1_score(preds_test, y_test)

0.8550407188991856

ЗАДАНИЕ 6.8

In [46]:
model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
f1_score(preds_test, y_test)

0.9376063216813916

ЗАДАНИЕ 6.9

In [48]:
model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring= f1_score,
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(


[CV 1/3] END ...learning_rate=1.0, n_estimators=1;, score=nan total time=   0.3s


c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(


[CV 2/3] END ...learning_rate=1.0, n_estimators=1;, score=nan total time=   0.2s


...

[CV 1/3] END learning_rate=0.010000000000000002, n_estimators=128;, score=nan total time=  52.9s


c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(


[CV 2/3] END learning_rate=0.010000000000000002, n_estimators=128;, score=nan total time=  36.3s


c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


[CV 3/3] END learning_rate=0.010000000000000002, n_estimators=128;, score=nan total time=  48.0s
Лучшие гиперпараметры: {'learning_rate': 1.0, 'n_estimators': 1}
Лучшее значение метрики: nan


ЗАДАНИЕ 6.10

In [50]:
model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.9574894279991099

ЗАДАНИЕ 6.11

In [56]:
model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
f1_score(preds_class, y_test)

Learning rate set to 0.068023
0:	learn: 0.6018254	total: 198ms	remaining: 3m 18s
1:	learn: 0.5117893	total: 245ms	remaining: 2m 2s
2:	learn: 0.4550684	total: 284ms	remaining: 1m 34s
3:	learn: 0.4109971	total: 320ms	remaining: 1m 19s
4:	learn: 0.3586409	total: 350ms	remaining: 1m 9s
5:	learn: 0.3283927	total: 370ms	remaining: 1m 1s
6:	learn: 0.3068417	total: 395ms	remaining: 56.1s
7:	learn: 0.2872791	total: 426ms	remaining: 52.9s
8:	learn: 0.2702372	total: 458ms	remaining: 50.4s
9:	learn: 0.2568044	total: 494ms	remaining: 48.9s
10:	learn: 0.2460252	total: 528ms	remaining: 47.5s
11:	learn: 0.2371697	total: 580ms	remaining: 47.7s
12:	learn: 0.2227435	total: 611ms	remaining: 46.4s
13:	learn: 0.2113300	total: 644ms	remaining: 45.4s
14:	learn: 0.2019338	total: 674ms	remaining: 44.3s
15:	learn: 0.1969024	total: 712ms	remaining: 43.8s
16:	learn: 0.1919289	total: 830ms	remaining: 48s
17:	learn: 0.1882538	total: 975ms	remaining: 53.2s
18:	learn: 0.1816902	total: 1.01s	remaining: 52.2s
19:	learn:

0.9605072866837246

ЗАДАНИЕ 6.12

In [59]:
cm = get_confusion_matrix(model, Pool(X_train, y_train))
print(cm)

[[46681.   525.]
 [ 1247. 34670.]]


ЗАДАНИЕ 6.13

In [61]:
clf = RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3,
                       random_state=31)

In [65]:
pd.DataFrame(
    {
        "feature_importance": clf.feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)


AttributeError: 'RandomForestClassifier' object has no attribute 'feature_importance'